In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from pytorch3d.renderer.cameras import FoVPerspectiveCameras, look_at_view_transform
from pytorch3d.renderer.camera_utils import join_cameras_as_batch
from pytorch3d.renderer.implicit.utils import ray_bundle_to_ray_points
from pytorch3d.renderer import VolumeRenderer, NDCMultinomialRaysampler, EmissionAbsorptionRaymarcher
from pytorch3d.structures import Pointclouds, Volumes, Meshes
from pytorch3d.ops import add_pointclouds_to_volumes, cubify

from nerv.renderer import make_cameras_dea

In [2]:
_device = torch.device("cpu")

In [3]:
batch_size = 2
n_channels = 4
shape = 8
n_pts_per_ray = 16
dtype=torch.float32

In [4]:
# Generate grid
zs = torch.linspace(-1, 1, steps=shape)
ys = torch.linspace(-1, 1, steps=shape)
xs = torch.linspace(-1, 1, steps=shape)
z, y, x = torch.meshgrid(zs, ys, xs)
grid = torch.stack([z, y, x], dim=-1) # torch.Size([100, 100, 100, 3])
vol_points = grid.unsqueeze(0).repeat(batch_size, 1, 1, 1, 1).reshape(batch_size, -1, 3)
# print(vol_points.shape)
vol_clouds = Pointclouds(vol_points)
# print(vol_clouds.get_bounding_boxes())

torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)


In [5]:
cameras = join_cameras_as_batch([
    make_cameras_dea(
        azim = +.15 * torch.ones(1, device=_device),
        elev = 0.00 * torch.ones(1, device=_device),
        dist = 4.00 * torch.ones(1, device=_device) 
    ),
    make_cameras_dea(
        azim = -.15 * torch.ones(1, device=_device),
        elev = 0.00 * torch.ones(1, device=_device),
        dist = 4.00 * torch.ones(1, device=_device) 
    ),   
])

print(cameras.R.shape)
print(cameras.T.shape)

raymarcher = EmissionAbsorptionRaymarcher()
raysampler = NDCMultinomialRaysampler(  
    image_width=shape,
    image_height=shape,
    n_pts_per_ray=n_pts_per_ray,  
    min_depth=2.0,
    max_depth=6.0,
)      

renderer = VolumeRenderer(
    raysampler=raysampler,
    raymarcher=raymarcher,
) 

torch.Size([2, 3, 3])
torch.Size([2, 3])


In [6]:
def get_frustum(cameras=None):
    batch_size = cameras.R.shape[0]
    features = torch.rand(
        size=[batch_size, n_channels, shape, shape, shape],
        device=_device,
        dtype=torch.float32,
    )

    densities = torch.ones(
        size=[batch_size, 1, shape, shape, shape],
        device=_device,
        dtype=torch.float32,
    )

    volumes = Volumes(
        features=features,
        densities=densities, 
        voxel_size= 3.0 / shape,
        volume_translation = [0, 0, 0],
    )
    
    _, ray_bundle = renderer(cameras=cameras, volumes=volumes) # [...,:3]
    ray_bundle = raysampler.forward(cameras=cameras, n_pts_per_ray=n_pts_per_ray)
    ray_points = ray_bundle_to_ray_points(ray_bundle).view(batch_size, -1, 3)  
    # print(ray_points.shape)
    ray_frustum = Pointclouds(ray_points)
    return ray_frustum

frustum0 = get_frustum(cameras=cameras[[0]])
frustum1 = get_frustum(cameras=cameras[[1]])

One or more elements of rays_densities are outside of validrange (0.0, 1.0)


In [7]:
verts = torch.tensor([[
    [+1.,+1.,+1.], [+1.,+1.,-1.],
    [-1.,+1.,-1.], [-1.,+1.,+1.],
    [+1.,-1.,+1.], [+1.,-1.,-1.],
    [-1.,-1.,-1.], [-1.,-1.,+1.]
]])

faces = torch.tensor([[
    [1., 0., 2.], [0., 3., 2.], 
    [2., 3., 6.], [7., 6., 3.], 
    [1., 4., 0.], [1., 5., 4.], 
    [1., 2., 6.], [1., 6., 5.], 
    [0., 4., 7.], [0., 7., 3.], 
    [4., 5., 7.], [4., 6., 7.], 
]])

vmesh = Meshes(verts, faces)

In [9]:
# Some of these imports are only needed for testing code coverage
from pytorch3d.vis.plotly_vis import (  # noqa: F401
    get_camera_wireframe,  # noqa: F401
    plot_batch_individually,  # noqa: F401
    plot_scene, 
    Lighting
)


fig = plot_scene(
    plots = {
        "Field of View": {
            # "ray_bundle": ray_bundle,
            # "fovcameras": cameras,
            # "ray_clouds": ray_clouds,
            # "vol_clouds": vol_clouds,
            "vmesh": vmesh,
            "cam0": cameras[[0]],
            "cam1": cameras[[1]],
            "ray0": frustum0, 
            "ray1": frustum1, 
        },
    }, 
    lighting=Lighting(
        ambient=0.1,
        diffuse=0.5,
        fresnel=0.0,
        specular=1.0,
        roughness=0.9,
        facenormalsepsilon=1e-6,
        vertexnormalsepsilon=1e-12,
    ),
    xaxis = {
        'range': [-4, 4],
        'showgrid': False, # thin lines in the background
        'zeroline': False, # thick line at x=0
        'visible': False,  # numbers below
    }, 
    yaxis = {
        'range': [-4, 4],
        'showgrid': False, # thin lines in the background
        'zeroline': False, # thick line at x=0
        'visible': False,  # numbers below
    }, 
    zaxis = {
        'range': [-4, 4],
        'showgrid': False, # thin lines in the background
        'zeroline': False, # thick line at x=0
        'visible': False,  # numbers below
    }, 
    pointcloud_marker_size = 2,
)
fig.update_layout(
    autosize=False,
    width=800,
    height=800,
)
fig.show()
